In [1]:
import json
import jsonlines
import statistics
from tqdm import tqdm
import openai
import re
openai.api_key = ""

In [2]:
data_1 = []
data_2 = []

path_1 = "Dialogizer"
path_2 = "WikiDialog_LLM_instruction_100"

with open(path_1 + '.json','r') as f: 
    data_1 = json.load(f)
with open(path_2 + '.json','r') as f: 
    data_2 = json.load(f)

In [3]:
relevant = []
well_formed = []
overall_quality = []

In [4]:
for i in range(3):
    print(f"instance {i}")
    context = ''
    instance_1 = data_1[i]
    instance_2 = data_2[i]
    
    if len(instance_1) != len(instance_2):
        raise
    
    for j in range(len(instance_1)):
        context += instance_1[j]['answer']
        
    for j in range(len(instance_1)):
        print(j)
        answer = instance_1[j]['answer']
        
        question_1 = instance_1[j]['question']
        question_2 = instance_2[j]['question']
        
        PROMPT= f"""
This is a task to evaluate the quality of a conversational question answering dataset. You
will be given [context, two candidate questions, answer], and your task is to compare the
quality of the candidate questions based on four criteria: contextual relevance, wellformedness, fluency, overall quality. For each criteria, answer which question is better.
1. Contextual Relevance: whether the question relevant to the answer/context
2. Well-formedness: whether the question is well-formed
3. Overall Quality: overall quality of the question
• Context: {context}
• Question A: {question_1}
• Question B: {question_2}
• Answer:
Choose the question which is more relevant to the given answer.
options: [Question A, Equal, Question B]
Choose the question which is more well-formed?
options: [Question A, Equal, Question B]
Choose the question which has better overall-quality.
options: [Question A, Equal, Question B]
        """

        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        result = response.choices[0].message.content
        
        
        
        try:
            
            
            
            
            matches = re.finditer(r'\[', result)


            # Find the indices of the second occurrence

            first_bracket = next(matches)
            second_bracket = next(matches)
            third_bracket = next(matches)


            matches = re.finditer(r'\]', result)

            # Find the indices of the second occurrence

            first_bracket_end = next(matches)
            second_bracket_end = next(matches)
            third_bracket_end = next(matches)


            relevant.append(result[first_bracket.start()+1:first_bracket_end.start()])
            well_formed.append(result[second_bracket.start()+1:second_bracket_end.start()])
            overall_quality.append(result[third_bracket.start()+1:third_bracket_end.start()])
        except:
            continue

        
        
    

instance 0
0
1
2
3
4
5
instance 1
0
1
2
3
4
5
instance 2
0
1
2
3
4
5


In [5]:
# check single prompt

PROMPT

'\nThis is a task to evaluate the quality of a conversational question answering dataset. You\nwill be given [context, two candidate questions, answer], and your task is to compare the\nquality of the candidate questions based on four criteria: contextual relevance, wellformedness, fluency, overall quality. For each criteria, answer which question is better.\n1. Contextual Relevance: whether the question relevant to the answer/context\n2. Well-formedness: whether the question is well-formed\n3. Overall Quality: overall quality of the question\n• Context: A furious Paul kills her.Meanwhile, Susan and Mike make peace by having sex for the first time.Mary Alice Young (Brenda Strong) is seen in this episode."The 59th Street Bridge Song (Feelin\' Groovy)" by Harpers Bizarre and Simon & Garfunkel is played during Lynette\'s disturbing dream.Originally the dream was to have Lynette driving through the country with her kids who were in the backseat and then proceeding to drive the car into a l

In [6]:
# check single result
result

'Contextual Relevance: Equal\nWell-formedness: Question B\nOverall Quality: Question B'

In [7]:
relevant

['Equal',
 'Question B',
 'Question B',
 'Equal',
 'Equal',
 'Question B',
 'Question A',
 'Equal',
 'Question B',
 'Equal',
 'Equal']

In [8]:
well_formed

['Equal',
 'Equal',
 'Equal',
 'Equal',
 'Equal',
 'Equal',
 'Equal',
 'Equal',
 'Equal',
 'Equal',
 'Question B']

In [9]:
overall_quality

['Equal',
 'Question B',
 'Question B',
 'Equal',
 'Equal',
 'Question B',
 'Question A',
 'Equal',
 'Question B',
 'Equal',
 'Question B']